# Pixelwise Segmentation

Use the `elf.segmentation` module for feature based instance segmentation from pixels.
Note that this example is educational and there are easier and better performing method for the image used here. These segmentation methods are very suitable for pixel embeddings learned with neural networks, e.g. with methods like [Semantic Instance Segmentation with a Discriminateive Loss Function](https://arxiv.org/abs/1708.02551).

## Image and Features

Load the relevant libraries. Then load an image from the skimage examples and compute per pixel features.

In [ ]:
%gui qt5
import time
import numpy as np

# import napari for data visualisation
import napari

# import vigra to compute per pixel features
import vigra

# elf segmentation functionality we need for the problem setup
import elf.segmentation.features as feats
from elf.segmentation.utils import normalize_input

In [ ]:
# we use the coins example image
from skimage.data import coins
image = coins()

In [ ]:
# We use blurring and texture filters from vigra.filters computed for different scales to obain pixel features.
# Note that it's certainly possible to compute better features for the segmentation problem at hand.
# But for our purposes, these features are good enough.
im_normalized = normalize_input(image)

scales = [4., 8., 12.]
image_features = [im_normalized[None]]  # use the normal image as 
for scale in scales:
    image_features.append(normalize_input(vigra.filters.gaussianSmoothing(im_normalized, scale))[None])
    feats1 = vigra.filters.hessianOfGaussianEigenvalues(im_normalized, scale)
    image_features.append(normalize_input(feats1[..., 0])[None])
    image_features.append(normalize_input(feats1[..., 1])[None])
    feats2 = vigra.filters.structureTensorEigenvalues(im_normalized, scale, 1.5 * scale)
    image_features.append(normalize_input(feats2[..., 0])[None])
    image_features.append(normalize_input(feats2[..., 1])[None])

image_features = np.concatenate(image_features, axis=0)
print("Feature shape:")
print(image_features.shape)

In [ ]:
# visualize the image and the features with napari
viewer = napari.Viewer()
viewer.add_image(im_normalized)
viewer.add_image(image_features)

## Segmentation Problem

Set up a graph segmentation problem based on the image and features with elf functionality.
To this end, we construct a grid graph and compute edge features for the inter pixel edges in this graph.

In [ ]:
# compute a grid graph for the image
shape = image.shape
grid_graph = feats.compute_grid_graph(shape)

# compute the edge features
# elf supports three different distance metrics to compute edge features
# from the image features:
# - 'l1': the l1 distance
# - 'l2': the l2 distance
# - 'cosine': the cosine distance (= 1. - cosine similarity)
# here, we use the l2 distance
distance_type = 'l2'

# 'compute_grid-graph-image_features' returns both the edges (=list of node ids connected by the edge)
# and the edge weights. Here, the edges are the same as grid_graph.uvIds()
edges, edge_weights = feats.compute_grid_graph_image_features(grid_graph, image_features, distance_type)

# we normalize the edge weigths to the range [0, 1]
edge_weights = normalize_input(edge_weights)

In [ ]:
# simple post-processing to ensure the background label is '0',
# filter small segments with a size of below 100 pixels
# and ensure that the segmentation ids are consecutive
def postprocess_segmentation(seg, shape, min_size=100):
    if seg.ndim == 1:
        seg = seg.reshape(shape)

    ids, sizes = np.unique(seg, return_counts=True)
    bg_label = ids[np.argmax(sizes)]

    if bg_label != 0:
        if 0 in seg:
            seg[seg == 0] = seg.max() + 1
        seg[seg == bg_label] = 0
    
    filter_ids = ids[sizes < min_size]
    seg[np.isin(seg, filter_ids)] = 0
    
    vigra.analysis.relabelConsecutive(seg, out=seg, start_label=1, keep_zeros=True)
    return seg

## Multicut

As the first segmentation method, we use Multicut segmentation, based on the grid graph and the edge weights we have just computed.

In [ ]:
# the elf multicut funtionality
import elf.segmentation.multicut as mc

In [ ]:
# In order to apply multicut segmentation, we need to map the edge weights from their initial value range [0, 1]
# to [-inf, inf]; where positive values represent attractive edges and negative values represent repulsive edges.

# When computing these "costs" for the multicut, we can set the threshold for when an edge is counted
# as repulsive with the so called boundary bias, or beta, parameter.
# For values smaller than 0.5 the multicut segmentation will under-segment more,
# for values larger than 0.4 it will over-segment more. 
beta = .75
costs = mc.compute_edge_costs(edge_weights, beta=beta)
print("Mapped edge weights in range", edge_weights.min(), edge_weights.max(), "to multicut costs in range", costs.min(), costs.max())

In [ ]:
# compute the multicut segmentation
t = time.time()
mc_seg = mc.multicut_kernighan_lin(grid_graph, costs)
print("Computing the segmentation with multicut took", time.time() - t, "s")
mc_seg = postprocess_segmentation(mc_seg, shape)

In [ ]:
# visualize the multicut segmentation
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(mc_seg)

## Long-range Segmentation Problem

For now, we have only taken "local" information into account for the segmentation problem.
More specifically, we have only solved the Multicut with edges derived from nearest neighbor pixel transitions.
Next, we will use two algorithms, Mutex Watershed and Lifted Multicut, that can take long range edges into account. This has the advantage that feature differences are often more pronounced along larger distances, thus yielding much better information with respect to label transition.
Here, we extract this information by defining a "pixel offset pattern" and comparing the pixel features for these offsets. For details about this segmentation approach check out [The Mutex Watershed: Efficient, Parameter-Free Image Partitioning](https://openaccess.thecvf.com/content_ECCV_2018/html/Steffen_Wolf_The_Mutex_Watershed_ECCV_2018_paper.html).

In [ ]:
# here, we define the following offset pattern:
# straight and diagonal transitions at a radius of 3, 9 and 27 pixels
# note that the offsets [-1, 0] and [0, -1] would correspond to the edges of the grid graph
offsets = [
    [-3, 0], [0, -3], [-3, 3], [3, 3],
    [-9, 0], [0, -9], [-9, 9], [9, 9],
    [-27, 0], [0, -27], [-27, 27], [27, 27]
]

# we have significantly more long range than normal edges.
# hence, we subsample the offsets, for which actual long range edges will be computed by setting a stride factor
strides = [2, 2]

distance_type = 'l2'  # we again use l2 distance
lr_edges, lr_edge_weights = feats.compute_grid_graph_image_features(grid_graph, image_features, distance_type,
                                                                    offsets=offsets, strides=strides,
                                                                    randomize_strides=False)
lr_edge_weights = normalize_input(lr_edge_weights)
print("Have computed", len(lr_edges), "long range edges, compared to", len(edges), "normal edges")

## Mutex Watershed

We use the Mutex Watershed to segment the image. This algorithm functions similar to (Lifted) Multicut, but is greedy and hence much faster. Despite its greedy nature, for many problems the solutions are of similar quality than Multicut segmentation.

In [ ]:
# elf mutex watershed functionality
import elf.segmentation.mutex_watershed as mws

In [ ]:
t = time.time()
mws_seg = mws.mutex_watershed_clustering(edges, lr_edges, edge_weights, lr_edge_weights)
print("Computing the segmentation with mutex watershed took", time.time() - t, "s")
mws_seg = postprocess_segmentation(mws_seg, shape)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(mws_seg)

## Lifted Multicut

Finally, we use Lifted Multicut segmentation. The Lifted Multicut is an extension to the Multicut, which can incorporate long range edges.

In [ ]:
# elf lifted multicut functionality
import elf.segmentation.lifted_multicut as lmc

In [ ]:
# For the lifted multicut, we again need to transform the edge weights in [0, 1] to costs in [-inf, inf]
beta = .75  # we again use a boundary bias of 0.75
lifted_costs = mc.compute_edge_costs(lr_edge_weights, beta=beta)

In [ ]:
t = time.time()
lmc_seg = lmc.lifted_multicut_kernighan_lin(grid_graph, costs, lr_edges, lifted_costs)
print("Computing the segmentation with lifted multicut took", time.time() - t, "s")
lmc_seg = postprocess_segmentation(lmc_seg, shape)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(lmc_seg)

## Comparing the segmentations

We can now compare the three different segmentation. Note that the comparison is not quite fair here, because we have used the beta parameter to bias the segmentation to more over-segmentation for Multicut and Lifted Multicut while applying the Mutex Watershed to unbiased edge weights.

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(mc_seg)
viewer.add_labels(mws_seg)
viewer.add_labels(lmc_seg)